<a href="https://colab.research.google.com/github/Zohairk4help/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-le

In [79]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar


--2022-01-23 13:13:38--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  5.69MB/s    in 0.2s    

2022-01-23 13:13:38 (5.69 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [80]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

# Load Amazon Data into Spark DataFrame

In [81]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
Musical_Instruments_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
Musical_Instruments_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

# Create DataFrame to match Tables

In [82]:
from pyspark.sql.functions import to_date
# Read in the review Dataset as a Dataframe

In [83]:
# Create the vine_table. DataFrame
vine_df = Musical_Instruments_df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])

vine_df.show()
vine_df.dtypes

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RMDCHWD0Y5OZ9|          3|            0|          1|   N|                N|
| RZSL0BALIYUNU|          5|            0|          0|   N|                Y|
| RIZR67JKUDBI0|          3|            0|          1|   N|                Y|
|R27HL570VNL85F|          5|            0|          0|   N|                Y|
|R34EBU9QDWJ1GD|          5|            0|          0|   N|                Y|
|R1WCUI4Z1SIQEO|          5|            0|          0|   N|                N|
| RL5LNO26GAVJ1|          2|            3|          4|   N|                Y|
|R3GYQ5W8JHP8SB|          5|            0|          0|   N|                Y|
|R30SHYQXGG5EYC|          5|            0|          0|   N|                Y|
|R14YLXA56NP51I|          5|            1|          1|   N|     

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string')]

In [84]:
# step 1: Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
vine_df1 = vine_df.filter("total_votes>=20")
vine_df1.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|  RMRD6SMF2AUQ|          3|            4|         21|   N|                N|
| RMPCXKWX3T57Y|          1|            1|         73|   N|                N|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|     

In [85]:
# step 2: Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows 
# where the number of helpful_votes divided by total_votes is equal to or greater than 50%.

import pyspark.sql.functions as F
from pyspark.sql.functions import *

vine_df2 = vine_df1.withColumn("helpful_percent",(F.col("helpful_votes") / F.col("total_votes")*100))
vine_df2.show()
#ref:https://stackoverflow.com/questions/53434077/create-a-new-column-with-filter
# ref:https://stackoverflow.com/questions/44033037/adding-constant-value-column-to-spark-dataframe

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|   helpful_percent|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N| 77.04918032786885|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y| 91.30434782608695|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y| 97.36842105263158|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|  89.1891891891892|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y| 95.83333333333334|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|  91.8918918918919|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N| 95.65217391304348|
|  RMRD6SMF2AUQ|          3|  

In [86]:
vine_df2a = vine_df2.filter("helpful_percent>=50")
vine_df2a.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|  helpful_percent|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|77.04918032786885|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|91.30434782608695|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|97.36842105263158|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N| 89.1891891891892|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|95.83333333333334|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y| 91.8918918918919|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|95.65217391304348|
|R1P7GJ0IN2BRNH|          5|           3

In [87]:
# step 3: Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows 
# where a review was written as part of the Vine program (paid), vine == 'Y'.

vine_Y_df3 = vine_df2a.filter(vine_df2a["vine"]=="Y")
vine_Y_df3.show()


+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|  helpful_percent|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R1R9RU7JW0MFR2|          4|           20|         23|   Y|                N|86.95652173913044|
|R19EFYNN3W8Q07|          5|           26|         32|   Y|                N|            81.25|
|R34DJ1R8AEU0SG|          5|           29|         35|   Y|                N|82.85714285714286|
|R25P5CXK5L9RHF|          5|          146|        161|   Y|                N| 90.6832298136646|
|R2E9VZB3I4LSN5|          5|           55|         59|   Y|                N|93.22033898305084|
| RKYLHZL7EPELX|          4|           19|         25|   Y|                N|             76.0|
|R1U13EKGQD3ZE6|          5|           22|         25|   Y|                N|             88.0|
| RYW05F1MUEF01|          5|           8

In [88]:
# step 4:Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.

vine_N_df3 = vine_df2a.filter(vine_df2a["vine"]=="N")
vine_N_df3.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|   helpful_percent|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N| 77.04918032786885|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y| 91.30434782608695|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y| 97.36842105263158|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|  89.1891891891892|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y| 95.83333333333334|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|  91.8918918918919|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N| 95.65217391304348|
|R1P7GJ0IN2BRNH|          5|  

# step 5:Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

reference: https://www.datasciencemadesimple.com/get-number-of-rows-and-number-of-columns-of-dataframe-in-pyspark/

In [89]:
# step 5:Determine the total number of reviews per Dfs

########## Get Size and shape of the dataframe[for the two types of review (paid vs unpaid)] in pyspark
total_unpaid_reviews = vine_N_df3.count()
total_paid_reviews = vine_Y_df3.count()

print((vine_N_df3.count(), " is the number of total unpaid reviews"))
print((vine_Y_df3.count(), "is the number of the total paid reviews"))

(14477, ' is the number of total unpaid reviews')
(60, 'is the number of the total paid reviews')


In [99]:
# step 5:Determine the number of 5-star reviews for the two types of review (paid vs unpaid).

########## Get the number of 5-star reviews from the dataframes[paid vs unpaid] in pyspark
vine_N_df3.groupBy("star_rating").agg(count(col("star_rating")==5)).show()
five_star_unpaid_reviews = vine_N_df3.filter(col("star_rating")==5).count()
print((five_star_unpaid_reviews))
print(( " 8212 is the number of total unpaid 5-star-reviews"))
# ref: https://stackoverflow.com/questions/49021972/pyspark-count-rows-on-condition

+-----------+------------------------+
|star_rating|count((star_rating = 5))|
+-----------+------------------------+
|          1|                    1532|
|          3|                    1292|
|          5|                    8212|
|          4|                    2687|
|          2|                     754|
+-----------+------------------------+

8212
 8212 is the number of total unpaid 5-star-reviews


In [101]:
# step 5:Determine the number of 5-star reviews for the two types of review (paid vs unpaid).

########## Get the number of 5-star reviews from the dataframes[paid vs unpaid] in pyspark
vine_Y_df3.groupBy("star_rating").agg(count(col("star_rating")==5)).show()
five_star_paid_reviews = vine_Y_df3.filter(col("star_rating")==5).count()
print((five_star_paid_reviews))
print(( " 34 is the number of total unpaid 5-star-reviews"))
# ref: https://stackoverflow.com/questions/49021972/pyspark-count-rows-on-condition

+-----------+------------------------+
|star_rating|count((star_rating = 5))|
+-----------+------------------------+
|          3|                       9|
|          5|                      34|
|          4|                      16|
|          2|                       1|
+-----------+------------------------+

34
 34 is the number of total unpaid 5-star-reviews


In [105]:
# step 5:Determine the percentage of 5-star reviews for the two types of review (paid vs unpaid).
percentage_unpaid_5_star_reviews = five_star_unpaid_reviews / total_unpaid_reviews * 100 
percentage_paid_5_star_reviews = five_star_paid_reviews / total_paid_reviews * 100
print (('Percentage of Unpaid 5-Star reviews:', percentage_unpaid_5_star_reviews))
print (('Percentage of Paid 5-Star reviews:', percentage_paid_5_star_reviews))

('Percentage of Unpaid 5-Star reviews:', 56.72445948746287)
('Percentage of Paid 5-Star reviews:', 56.666666666666664)
